<a href="https://colab.research.google.com/github/guilhermelaviola/ApplicationsOfDataScienceInDisruptiveTechnologies/blob/main/Class13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sparse Optical Flow**
Sparse optical flow is a computer vision technique focused on tracking selected points of interest in video sequences, offering advantages in computational efficiency and accuracy for real-time applications like object tracking, traffic monitoring, and sports motion analysis. The Lucas-Kanade algorithm is a popular method for computing this technique, relying on the assumption of relatively constant pixel movement in small regions between frames. Key steps include detecting interest points with the Harris corner or Shi-Tomasi detectors, and utilizing the Lucas-Kanade algorithm to derive motion vectors indicating direction and speed. Optimizing the algorithm is vital for robust tracking in challenging conditions, often combining it with predictive filters such as the Kalman Filter to maintain tracking during occlusions or erratic movements. Sparse optical flow is a versatile tool for motion analysis across various fields, balancing efficiency and accuracy effectively.

In [1]:
# Importing all the necessary libraries:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [2]:
# Uploading a video from the Colab library:
video_path = 'https://github.com/opencv/opencv/blob/master/samples/data/vtest.avi?raw=true'
cap = cv2.VideoCapture(video_path)

# Defining parameters to detect points of interest:
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Reading the first panel:
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Detecting points of interest:
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

In [ ]:
# Calculating the Optical Flow:
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Creating a mask to draw the trajectories:
mask = np.zeros_like(old_frame)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculating optical flux:
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Selecting good points:
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Drawing the points and their trajectories:
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)  # Converting to integer
        c, d = old.ravel().astype(int)   # Converting to integer
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
        frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)
    cv2_imshow(img)

    # Updating frames and points:
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()